In [1]:
!pip install pyspark py4j #I found this way of installation more "clean"

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 199 kB 50.4 MB/s 
     |████████████████████████████████| 198 kB 60.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2f566d37610edca85efd61516ccfa6613af54eb04be6d788c645f0217ba6b706
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import os

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar > /dev/null

--2022-05-02 19:46:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.98MB/s    in 0.2s    

2022-05-02 19:46:12 (4.98 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [8]:
# Load in data from S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [10]:
# filter for only columns with 20 or more total votes 
cleaned_vine_df = vine_df.filter('total_votes>=20')
cleaned_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|ROP6ITXO8K5V2 |5          |23           |27         |N   |Y                |
|R3ND1LVU7AXCVF|1          |21           |21         |N   |Y                |
|R9I5FOLKU99RY |5          |19           |20         |Y   |N                |
|R1QS8AOD6HX3ED|4          |59           |81         |N   |N                |
|R3ED60RC69CCQ6|5          |22           |23         |Y   |N                |
|R2MDI1TP46VSYD|3          |9            |21         |N   |Y                |
|R2JM687C525WR9|3          |33           |33         |Y   |N                |
|R2LWX4TZ67FWPT|4          |50           |50         |N   |N                |
|R29IYHPYD14AGI|3          |84           |84         |N   |Y                |
|R1F2I723WRK5QV|5          |20           |20         |N   |Y    

In [11]:
# filter for only columns with mostly helpful votes
hvine_df = cleaned_vine_df.filter("helpful_votes/total_votes>=0.5")
hvine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|     

In [12]:
vine_program_df = hvine_df[hvine_df.verified_purchase == 'Y']
print(vine_program_df.count())
vine_program_df.show()

29583
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R18T8SFBDXAKSY|          5|           95|         95|   N|                Y|
|R2TETIV0IVWFSJ|          4|           80|         82|   N|                Y|
|R1VBVD23GPZW0E|          5|           55|         58|   N|                Y|
|R1AJKXVHURPVCZ|          3|           12|         22|   N|                Y|
|R32RD311POLW7G|          3|           25|         27|   N

In [13]:
not_vine_program_df = hvine_df[hvine_df.verified_purchase == 'N']
print(not_vine_program_df.count())
not_vine_program_df.show()

33711
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|                N|
|R2F7D9N3SSQC20|          4|           32|         34|   N|                N|
| RN4VK9FV66YPS|          5|           29|         33|   N|                N|
|R36SKVJXZ0I98A|          5|          124|        127|   N

In [15]:
five_star_paid_reviews = vine_program_df[vine_program_df.star_rating == 5].count()
not_five_star_paid_reviews = vine_program_df[vine_program_df.star_rating != 5].count()
percentage = (five_star_paid_reviews / not_five_star_paid_reviews) * 100
print('Number of 5 stars: ' + str(five_star_paid_reviews), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 14324 Percentage: 93.87


In [17]:
five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating == 5].count()
not_five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating != 5].count()
percentage = (five_star_unpaid / not_five_star_unpaid) * 100
print('Number of 5 stars: ' + str(five_star_unpaid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 16090 Percentage: 91.31
